In [ ]:
#MAC USERS: 
#Before running ipynb - command line:
#ray stop
#ray start --head --port=6379 --redis-password="cbgt2"
#ray start --address='10.162.14.77:6379' --redis-password='cbgt2'
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload 
#if it doesn't work, shut down the ray server and start it again

In [ ]:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct_copy.py
#!autopep8 --in-place --aggressive --aggressive generateepochs.py

In [1]:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct_copy as popconstruct
import generateepochs as gen
import pdb

/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-08-13 10:09:11,216	INFO worker.py:727 -- Connecting to existing Ray cluster at address: 10.226.209.114:6379


# Reward schedule

In [2]:
def GenRewardSchedule(n_trials, volatility, conflict, reward_mu, reward_std):
    
    reward_t1, reward_t2 = gen.define_reward(conflict, n_trials, reward_mu, reward_std)
    cp_idx, cp_indicator = gen.define_changepoints(n_trials, reward_t1, reward_t2, volatility)
    t1_epochs, t2_epochs, noisy_pattern, volatile_pattern = gen.define_epochs(n_trials, reward_t1, reward_t2, cp_idx, conflict)
    return volatile_pattern, cp_idx, cp_indicator, noisy_pattern, t1_epochs, t2_epochs
    

In [3]:
rsg = cbgt.Pipeline() # rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t1_epochs,
 rsg.t2_epochs,
) = rsg[GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std,
    ).shape(6)

In [4]:
GenRewardSchedule(5, 2, 0.7, 1, 0.1)

(array([0., 0., 1., 0., 1.]),
 [0, 2, 3, 4],
 array([1., 0., 1., 1., 1.]),
 [0.0, 1.0, 1.0, 1.0, 0.0],
 array([0.        , 0.9564879 , 1.09041642, 1.08438992, 0.        ]),
 array([1.06740757, 0.        , 0.        , 0.        , 0.98670123]))

# Net parameters 

In [5]:
pl = cbgt.Pipeline()

In [6]:
pl.add(rsg)

In [7]:
pl.trial_num=0 # We need to figure out how to update this , maybe in a loop ?
# chosen action from somewhere, depending on last trial
pl.chosen_action = 2


In [8]:
#MODIFIERS - init_params.py 

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels = par.helper_actionchannels(self.channels)
    
#POPDATA + PATHWAYS - popconstruct_copy.py

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)
    
def codeblock_poppathways(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

#CONNECTIVITY GRIDS - popconstruct_copy.py

def codeblock_connAMPA(self):
    self.connectivity_AMPA, self.meaneff_AMPA = popconstruct.helper_connectivityAMPA(self.popdata, self.pathways)

def codeblock_connGABA(self): 
    self.connectivity_GABA, self.meaneff_GABA = popconstruct.helper_connectivityGABA(self.popdata, self.pathways) 

def codeblock_connNMDA(self): 
    self.connectivity_NMDA, self.meaneff_NMDA = popconstruct.helper_connectivityNMDA(self.popdata, self.pathways) 

def codeblock_get_reward_val(self):
    self.reward_val = par.get_reward_value(rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num)
    
    
# Q VALUE DATAFRAME INIT AND UPDATE
def codeblock_init_Q_support_params(self):
    self.Q_support_params = par.helper_init_Q_support_params()

def codeblock_update_Q_support_params(self,reward_val,chosen_action):
    self.Q_support_params = par.helper_update_Q_support_params(self.Q_support_params,self.reward_val,pl.chosen_action)
    
def codeblock_init_Q_df(self):
    self.Q_df = par.helper_init_Q_df(self.channels)
    
def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = par.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults,pl.trial_num)
    


In [9]:
#DEFAULTS - init_params.py
pl.celldefaults = pl[par.helper_cellparams]()
pl.popspecific = pl[par.helper_popspecific]()
pl.receptordefaults = pl[par.helper_receptor]()
pl.basestim = pl[par.helper_basestim]()
pl.dpmndefaults = pl[par.helper_dpmn]()
pl.d1defaults = pl[par.helper_d1]()
pl.d2defaults = pl[par.helper_d2]()
#pl.actionchannels = pl[par.helper_actionchannels]()
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

# Initiating the q_value pipeline
q_val_pipe = cbgt.Pipeline()
q_val_pipe.Q_support_params = q_val_pipe[par.helper_init_Q_support_params]()
rsg.reward_val = q_val_pipe[par.get_reward_value](rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num) 
q_val_pipe.Q_support_params = q_val_pipe[par.helper_update_Q_support_params](q_val_pipe.Q_support_params,rsg.reward_val,pl.chosen_action)

q_val_pipe.Q_df = q_val_pipe[par.helper_init_Q_df](pl.actionchannels)
(q_val_pipe.Q_df, q_val_pipe.Q_support_params, pl.dpmndefaults) = q_val_pipe[par.helper_update_Q_df](q_val_pipe.Q_df,q_val_pipe.Q_support_params,pl.dpmndefaults,pl.trial_num).shape(3)


# Adding the two pipelines to the main pipeline
pl.add(q_val_pipe)



In [10]:
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)
pl.add(codeblock_connAMPA)
pl.add(codeblock_connGABA)
pl.add(codeblock_connNMDA)

In [11]:
#DEFAULT VALUES 

environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']),
    'newpathways' : None,
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
    'chosen_action':1, #channel label
    'trial_num': 1, # goes from 0 to number of trials
    

}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)


(pid=317570) helper_init_Q_support_params
(pid=317570)   bayes_unif_min bayes_unif_max  ... reward_value chosen_action
(pid=317570) 0           0.0`           2.0`  ...        -1.0`            1`
(pid=317570) 
(pid=317570) [1 rows x 8 columns]
(pid=317570) get_reward_value
(pid=317570) 0.0
(pid=317570) helper_update_Q_support_params
(pid=317570)    bayes_unif_min  bayes_unif_max  ...  reward_value  chosen_action
(pid=317570) 0             0.0             2.0  ...           0.0              2
(pid=317570) 
(pid=317570) [1 rows x 8 columns]
(pid=317570) helper_init_Q_df
(pid=317570) num_actions 2
(pid=317570) Q_df Empty DataFrame
(pid=317570) Columns: [1, 2]
(pid=317570) Index: []
(pid=317570) Q_df      1    2
(pid=317570) 0  0.5  0.5
(pid=317570) In update_Q_df
(pid=317570) Q_support_params
(pid=317570)    bayes_unif_min  bayes_unif_max  ...  reward_value  chosen_action
(pid=317570) 0             0.0             2.0  ...           0.0              2
(pid=317570) 
(pid=317570) [1 rows x 

In [12]:
results['t1_epochs']

array([0.90500202, 0.        , 0.98997925, 0.        , 1.09670284])

In [13]:
locals().update(results)

In [14]:
results['Q_df']

,1,2
0,0.5,0.500
0,0.5,0.275


In [15]:
results['t1_epochs']

array([0.90500202, 0.        , 0.98997925, 0.        , 1.09670284])

In [16]:
results['dpmndefaults']

,dpmn_tauDOP,dpmn_alpha,dpmn_DAt,dpmn_taum,dpmn_dPRE,dpmn_dPOST,dpmn_tauE,dpmn_tauPRE,dpmn_tauPOST,dpmn_wmax,dpmn_w,dpmn_Q1,dpmn_Q2,dpmn_m,dpmn_E,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST
0,2`,0.3`,0.0`,1e+100`,0.8`,0.04`,15`,15`,6`,0.3`,0.1286`,0.0`,0.0`,1.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`


In [17]:
results['Q_support_params']

,bayes_unif_min,bayes_unif_max,bayes_H,bayes_sF,q_alpha,dpmn_CPP_scale,reward_value,chosen_action
0,0.0,2.0,0.05,1.25,0.45,15.0,0.0,2
